In [1]:
def read_file(filepath):
    with open(filepath) as f:
        str_text = f.read()
        
    return str_text

In [3]:
#read_file('moby_dick_four_chapters.txt')

In [4]:
import spacy

In [12]:
# changed module 'en' to fullname due to spacy error
nlp = spacy.load('en_core_web_sm',disable=['parser','tagger','ner','lemmatizer'])

In [13]:
nlp.max_length = 1198623

In [14]:
def separate_punc(doc_text):
    return [token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']

In [15]:
d = read_file('moby_dick_four_chapters.txt')

In [16]:
tokens = separate_punc(d)

In [11]:
len(tokens)

11338

In [18]:
#tokens

In [19]:
# pass in 25 words ==> network predict #26

In [20]:
train_len = 25 + 1

text_sequences = []

for i in range(train_len,len(tokens)):
    seq = tokens[i-train_len:i]
    
    text_sequences.append(seq)

In [21]:
type(text_sequences)

list

In [26]:
#text_sequences[1]

In [27]:
' '. join(text_sequences[0])

'call me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on'

In [28]:
from keras.preprocessing.text import Tokenizer

In [29]:
tokenizer = Tokenizer()

In [31]:
tokenizer.fit_on_texts(text_sequences)

In [32]:
sequences = tokenizer.texts_to_sequences(text_sequences)

In [39]:
# for i in sequences[0]:
#     print(f'{i} : {tokenizer.index_word[i]}')

In [33]:
vocabulary_size = len(tokenizer.word_counts)

In [34]:
vocabulary_size

2718

In [35]:
import numpy as np

In [36]:
sequences = np.array(sequences)

In [37]:
sequences

array([[ 956,   14,  263, ..., 2713,   14,   24],
       [  14,  263,   51, ...,   14,   24,  957],
       [ 263,   51,  261, ...,   24,  957,    5],
       ...,
       [ 952,   12,  166, ...,  262,   53,    2],
       [  12,  166, 2712, ...,   53,    2, 2718],
       [ 166, 2712,    3, ...,    2, 2718,   26]])

In [44]:
from tensorflow.keras.utils import to_categorical

In [48]:
X = sequences[:,:-1]

In [49]:
y = sequences[:,-1]

In [50]:
y = to_categorical(y,num_classes=vocabulary_size+1)

In [51]:
seq_len = X.shape[1]

In [52]:
X.shape

(11312, 25)

In [53]:
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding

In [56]:
def create_model(vocabulary_size,seq_len):
    
    model = Sequential()
    model.add(Embedding(vocabulary_size,seq_len,input_length=seq_len))
    model.add(LSTM(50,return_sequences=True))
    model.add(LSTM(50))
    model.add(Dense(50,activation='relu'))
    
    model.add(Dense(vocabulary_size, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
   
    model.summary()
    
    return model

In [60]:
model = create_model(vocabulary_size+1,seq_len)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 25, 25)            67975     
                                                                 
 lstm_2 (LSTM)               (None, 25, 50)            15200     
                                                                 
 lstm_3 (LSTM)               (None, 50)                20200     
                                                                 
 dense_2 (Dense)             (None, 50)                2550      
                                                                 
 dense_3 (Dense)             (None, 2719)              138669    
                                                                 
Total params: 244,594
Trainable params: 244,594
Non-trainable params: 0
_________________________________________________________________


In [58]:
from pickle import dump,load

In [61]:
model.fit(X,y,batch_size=128,epochs=2,verbose=1)

Epoch 1/2
89/89 [==============================] - 6s 48ms/step - loss: 7.0031 - accuracy: 0.0393
Epoch 2/2
89/89 [==============================] - 3s 39ms/step - loss: 6.3780 - accuracy: 0.0529


In [62]:
model.save('my_mobydick_model.h5')

In [63]:
dump(tokenizer,open('my_simpletokenizer','wb'))

In [64]:
from keras.preprocessing.sequence import pad_sequences

In [78]:
# Generate new text

def generate_text(model,tokenizer,seq_len,seed_text,num_gen_words):
    
    output_text = []
    
    input_text = seed_text
    
    for i in range(num_gen_words):
        
        # Take the input text string and encode it to a sequence
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        
        # Pad sequences to our trained rate (50 words in the video)
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
        
        # Predict Class Probabilities for each word
        pred_word_ind = model.predict(pad_encoded, verbose=0)[0]
        
        # Grab word
        pred_word = tokenizer.index_word[pred_word_ind] 
        
        # Update the sequence of input text (shifting one over with the new word)
        input_text += ' ' + pred_word
        
        output_text.append(pred_word)
        
    # Make it look like a sentence.
    return ' '.join(output_text)

In [67]:
seq_len

25

In [85]:
text_sequences[0]

['call',
 'me',
 'ishmael',
 'some',
 'years',
 'ago',
 'never',
 'mind',
 'how',
 'long',
 'precisely',
 'having',
 'little',
 'or',
 'no',
 'money',
 'in',
 'my',
 'purse',
 'and',
 'nothing',
 'particular',
 'to',
 'interest',
 'me',
 'on']

In [86]:
import random
random.seed(101)
random_pick = random.randint(0,len(text_sequences))

In [87]:
random_seed_text = text_sequences[random_pick]

In [88]:
#random_seed_text

In [89]:
seed_text = ' '.join(random_seed_text)

In [90]:
seed_text

"thought i to myself the man 's a human being just as i am he has just as much reason to fear me as i have"

In [91]:
generate_text(model,tokenizer,seq_len,seed_text=seed_text,num_gen_words=50)

TypeError: unhashable type: 'numpy.ndarray'

In [92]:
pred_word_ind.shape

NameError: name 'pred_word_ind' is not defined